# Integration of Limiting Factors study into OpenAirClim

Description: Comparison of the AirClim 2.1 contrail method with the new contrail method based on Megill & Grewe (2025).

Author: Liam Megill

Institution: Deutsches Zentrum für Luft- und Raumfahrt, Institut für Physik der Atmosphäre

We start by comparing the pre-calculated contrail data - $p_{SAC}$ in AirClim; $p_{pcf}$ in OpenAirClim. From the new CCMod simulations, it was possible to create pre-calculated data for a conventional aircraft as well, which is saved as `resp_cont_cc.nc`. Since a number of simulations are at the time of writing still missing, it was unfortunately not possible to create this data for the hydrogen-powered aircraft.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import numpy as np

# load data
ds_ah = xr.open_dataset("../../oac/repository/resp_cont.nc")
ds_cc = xr.open_dataset("../../oac/repository/resp_cont_cc.nc")
ds_lf = xr.open_dataset("../../oac/repository/resp_cont_lf.nc")

In [ ]:
# regrid AHEAD to ERA5 pressure levels

def calc_airclim_source_grid(ds_airclim):
    source_lvl = ds_airclim.plev.data
    source_lat = ds_airclim.lat.data
    source_lon = ds_airclim.lon.data
    lvl_source_mesh, lat_source_mesh, lon_source_mesh = np.meshgrid(
        source_lvl, source_lat, source_lon, indexing='ij'
    )
    source_points = np.column_stack(
        [lvl_source_mesh.flatten(),
         lat_source_mesh.flatten(),
         lon_source_mesh.flatten()]
    )
    return source_points

def regrid_to_target(ds, source_points, variable, target_lvl, target_lat, target_lon,
                     method="nearest"):

    # define target grid
    lvl_mesh, lat_mesh, lon_mesh = np.meshgrid(target_lvl, target_lat, target_lon, indexing='ij')
    target_points = np.column_stack(
        [lvl_mesh.flatten(),
         lat_mesh.flatten(),
         lon_mesh.flatten()]
    )

    # regrid variable
    intrp = griddata(
        points=source_points,
        values=ds[variable].data.flatten(),
        xi=target_points,
        method=method
    )
    intrp = intrp.reshape(lvl_mesh.shape)

    # convert to dataset
    ds_intrp = xr.DataArray(
        intrp,
        coords={"plev": target_lvl, "lat": target_lat, "lon": target_lon},
        dims=["plev", "lat", "lon"]
    )

    return ds_intrp


# define target grid
target_lvl = ds_lf.plev.data
target_lat = ds_ah.lat.data
target_lon = ds_ah.lon.data
target = (target_lvl, target_lat, target_lon)

# regrid data
ds_ah = ds_ah.transpose("plev", "lat", "lon")
ah_source_pts = calc_airclim_source_grid(ds_ah)
ds_ah_intrp_con = regrid_to_target(ds_ah, ah_source_pts, "SAC_CON", *target)
ds_ah_intrp_h2c = regrid_to_target(ds_ah, ah_source_pts, "SAC_LH2", *target)

In [ ]:
# show level values
levels = [350, 300, 250, 225, 200, 175, 150]
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
labels = ["Megill_2025", "CCMod", "AHEAD"]

ds_lst = [ds_lf.ppcf.sel(AC="oac4"), ds_cc.SAC_CON, ds_ah_intrp_con]
ds_latg_lst = [ds.groupby("lat").mean(dim="lon") for ds in ds_lst]

fig, axs = plt.subplots(len(levels), figsize=(7, len(levels)), sharex="all", sharey="all")
for j, ax in enumerate(axs[::-1]):
    for i, ds_latg in enumerate(ds_latg_lst):
        ax.plot(ds_latg.lat.data, ds_latg.sel(plev=levels[j]).data, c=colors[i], label=labels[i])
    
    # axis settings
    ax.set_xlim([-90, 90])
    ax.set_xticks([-90, -60, -30, 0, 30, 60, 90])
    ax.grid(visible=True, which="major", lw=0.1, c="gray", alpha=0.2)
    lvl_ax = ax.twinx()
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{levels[j]:.0f} hPa"])

# axis labels
axs[3].set_ylabel("Potential persistent contrail formation $p_{PCF}$ [-]")
axs[-1].set_xlabel("Latitude [deg]")
axs[-1].legend(loc="upper center", ncol=3)
fig.suptitle("Conventional aircraft $p_{pcf}$")
fig.tight_layout()
plt.subplots_adjust(hspace=0, wspace=0.175)
# fig.savefig("figs/comp_latg_con.png", dpi=250)

In [ ]:
# show level values
levels = [350, 300, 250, 225, 200, 175, 150]
colors = ['#2271B2', '#F748A5']
labels = ["Megill_2025", "AHEAD"]

ds_lst = [ds_lf.ppcf.sel(AC="oac10"), ds_ah_intrp_h2c]
ds_latg_lst = [ds.groupby("lat").mean(dim="lon") for ds in ds_lst]

fig, axs = plt.subplots(len(levels), figsize=(7, len(levels)), sharex="all", sharey="all")
for j, ax in enumerate(axs[::-1]):
    for i, ds_latg in enumerate(ds_latg_lst):
        ax.plot(ds_latg.lat.data, ds_latg.sel(plev=levels[j]).data, c=colors[i], label=labels[i])
    
    # axis settings
    ax.set_xlim([-90, 90])
    ax.set_xticks([-90, -60, -30, 0, 30, 60, 90])
    ax.grid(visible=True, which="major", lw=0.1, c="gray", alpha=0.2)
    lvl_ax = ax.twinx()
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{levels[j]:.0f} hPa"])

# axis labels
axs[3].set_ylabel("Potential persistent contrail formation $p_{PCF}$ [-]")
axs[-1].set_xlabel("Latitude [deg]")
axs[-1].legend(loc="upper center", ncol=3)
fig.suptitle("Hydrogen combustion aircraft $p_{pcf}$")
fig.tight_layout()
plt.subplots_adjust(hspace=0, wspace=0.175)
# fig.savefig("figs/comp_latg_h2c.png", dpi=250)

## Historical aviation scenario (conventional fuel)

We now run OpenAirClim with a historical aviation scenario. Here, I'm using an ELK emission inventory in the year 2019 with a normalisation for global air traffic from 1920 until 2019.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# load results
ds_res_mg = xr.open_dataset("../results/megill/hist/hist.nc")
ds_res_ac = xr.open_dataset("../results/ahead/hist/hist.nc")
ds_res_cc = xr.open_dataset("../results/ccmod/hist/hist.nc")

# load contrail export files
cont_exp_mg = xr.open_dataset("../results/megill/hist/cont_export.nc")
cont_exp_ac = xr.open_dataset("../results/ahead/hist/cont_export.nc")
cont_exp_cc = xr.open_dataset("../results/ccmod/hist/cont_export.nc")

# configure colours
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

Now, let us have a look at the contrail radiative forcing for each contrail module.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))

# contrail RF
ax.plot(ds_res_mg.time, ds_res_mg.RF_cont * 1e3, c=colors[0], label="Megill_2025")
ax.plot(ds_res_cc.time, ds_res_cc.RF_cont * 1e3, c=colors[1], label="CCMod")
ax.plot(ds_res_ac.time, ds_res_ac.RF_cont * 1e3, c=colors[2], label="AHEAD")
ax.legend(loc="best")
ax.set_xlabel("Year")
ax.set_ylabel("Contrail Radiative Forcing [mW/m2]")

fig.tight_layout()

ds_res_cc.RF_cont.data[-1] / ds_res_ac.RF_cont.data[-1]
# fig.savefig("figs/comp_hist_rf_con.png")

In [ ]:
fig, axs = plt.subplots(figsize=(9, 4), ncols=2)

# coverage
axs[0].plot(cont_exp_mg.lat, cont_exp_mg.sel(year=2019).cccov_tot, c=colors[0], label="Megill_2025")
axs[0].plot(cont_exp_cc.lat, cont_exp_cc.sel(year=2019).cccov_tot, c=colors[1], label="CCMod")
axs[0].plot(cont_exp_ac.lat, cont_exp_ac.sel(year=2019).cccov_tot, c=colors[2], label="AHEAD")
axs[0].legend(loc="best")
axs[0].set_xlabel("Latitude [deg]")
axs[0].set_ylabel("Total contrail cirrus coverage")

# contrail RF
axs[1].plot(ds_res_mg.time, ds_res_mg.RF_cont * 1e3, c=colors[0], label="Megill_2025")
axs[1].plot(ds_res_cc.time, ds_res_cc.RF_cont * 1e3, c=colors[1], label="CCMod")
axs[1].plot(ds_res_ac.time, ds_res_ac.RF_cont * 1e3, c=colors[2], label="AHEAD")
axs[1].set_xlabel("Year")
axs[1].set_ylabel("Contrail Radiative Forcing [mW/m2]")

fig.tight_layout()

We can take a closer look at the CFDD and cccov values

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5), subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

cfdd_diff_mg = (cont_exp_mg.cfdd - cont_exp_ac.cfdd)
im = cfdd_diff_mg.plot(ax=ax, add_colorbar=False)
fig.colorbar(im, fraction=0.024, pad=0.09, label="")
ax.set_title("Difference in CFDD, Megill_2025 vs. AHEAD")
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
# fig.savefig("figs/comp_hist_cfdd_con.png")

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5), subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

cccov_diff_mg = (cont_exp_mg.cccov - cont_exp_ac.cccov) * 100.0
im = cccov_diff_mg.plot(ax=ax, add_colorbar=False)
fig.colorbar(im, fraction=0.024, pad=0.09, label="Difference in cccov [%]")
ax.set_title("Difference in cccov, Megill_2025 vs. AHEAD")
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
# fig.savefig("figs/comp_hist_cccov_con.png")

## Historical aviation scenario (hydrogen)

We now run the same aviation scenario, but this time assuming only hydrogen combustion aircraft. I did not modify the `PM_rel` parameter, so the actual values are not accurate, but they can still be used to compare the methods.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# load results
ds_res_mg = xr.open_dataset("../results/megill/hist_h2c/hist_h2c.nc")
ds_res_ac = xr.open_dataset("../results/ahead/hist_h2c/hist_h2c.nc")

# load contrail export files
cont_exp_mg = xr.open_dataset("../results/megill/hist_h2c/cont_export.nc")
cont_exp_ac = xr.open_dataset("../results/ahead/hist_h2c/cont_export.nc")

# configure colours
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5), subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

cccov_diff_mg = (cont_exp_mg.cccov - cont_exp_ac.cccov) * 100.0
im = cccov_diff_mg.plot(ax=ax, add_colorbar=False)
fig.colorbar(im, fraction=0.024, pad=0.09, label="Difference in cccov [%]")
ax.set_title("Difference in cccov, Megill_2025 vs. AHEAD")
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
# fig.savefig("figs/comp_hist_cccov_h2c.png")

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(ds_res_mg.time, ds_res_mg.RF_cont * 1e3, c=colors[0], label="Megill_2025")
ax.plot(ds_res_ac.time, ds_res_ac.RF_cont * 1e3, c=colors[2], label="AHEAD")
ax.legend(loc="best")
ax.set_xlabel("Year")
ax.set_ylabel("Contrail Radiative Forcing [mW/m2]")
fig.tight_layout()
# fig.savefig("figs/comp_hist_rf_h2c.png")

In [ ]:
fig, ax = plt.subplots()
ax.plot(cont_exp_mg.lat, cont_exp_mg.sel(year=2019).cccov_tot, c=colors[0], label="Megill_2025")
ax.plot(cont_exp_ac.lat, cont_exp_ac.sel(year=2019).cccov_tot, c=colors[2], label="AHEAD")
ax.legend(loc="best")
ax.set_xlabel("Latitude [deg]")
ax.set_ylabel("Total contrail cirrus coverage")

## Historical aviation scenario (Low G)

Finally, we consider another fictional scenario, where all aircraft are assumed to be powered by a engines with low $G$ values. As before, we do not modify the `PM_rel` parameter.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# load results
ac_lst = ["g0p50", "g0p75", "g1p00", "g1p25", "g1p50", "g1p75"]
ac_g_vals = ["0.50", "0.75", "1.00", "1.25", "1.50", "1.75"]
ds_res_lst = [
    xr.open_dataset(f"../results/megill/hist_{ac}/hist_{ac}.nc") for ac in ac_lst
]

# configure colours
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))
for i_ac, ac in enumerate(ac_lst):
    ax.plot(
        ds_res_lst[i_ac].time, ds_res_lst[i_ac].RF_cont * 1e3,
        c=colors[i_ac], label=f"$G$ = {ac_g_vals[i_ac]} Pa/K"
    )
ax.legend(loc="best")
ax.set_xlabel("Year")
ax.set_ylabel("Contrail Radiative Forcing [mW/m2]")
fig.tight_layout()
# fig.savefig("figs/comp_hist_rf_lowg.png")